In [1]:
##changing the working dierc 
import os
os.chdir('P:\SAMPLE DATA')


In [2]:
import pandas as pd

In [1]:
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [4]:
###importing necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re


ModuleNotFoundError: No module named 'tensorflow'

In [18]:
df = pd.read_excel("nps responses.xlsx")

In [19]:
df.head()

,comment
0,- o been waiting for 9 months for one of your ...
1,*Useful admin support\n* Competitive purchasin...
2,.
3,.
4,.


In [20]:
df.isnull().sum()

comment    0
dtype: int64

In [21]:
# Function to Clean the Tweet.
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|([RT])', ' ', tweet.lower()).split())

In [24]:
 df['comment'] = df['comment'].apply(lambda x : clean_tweet(x))

In [25]:
df.head()

,comment
0,o been waiting for 9 months for one of your su...
1,useful admin support competitive purchasing pa...
2,
3,
4,


In [26]:
df['comment'].replace('', np.nan, inplace=True)

In [27]:
df.isnull().sum()

comment    8
dtype: int64

In [28]:
df.dropna(inplace=True)

In [29]:
def vader_sentiment_result(sent):
    scores = analyzer.polarity_scores(sent)
    
    if scores['compound'] >= 0.05 :
        return "Positive"
 
    elif scores['compound'] <= - 0.05 :
        return "Negative"
 
    else :
        return "Neutral"

In [30]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [31]:
df["vader_result"] = df["comment"].apply(lambda x: vader_sentiment_result(x))

In [32]:
df.head()

,comment,vader_result
0,o been waiting for 9 months for one of your su...,Neutral
1,useful admin support competitive purchasing pa...,Positive
10,because of a deeply disappointing experience w...,Negative
11,10 a week membership and not always the best v...,Negative
12,1m lost on straw trading to which all members ...,Negative


In [33]:
df['vader_result'] = df['vader_result'].replace('Neutral',2).replace('Positive',0).replace('Negative',1)

In [34]:
df.head()

,comment,vader_result
0,o been waiting for 9 months for one of your su...,2
1,useful admin support competitive purchasing pa...,0
10,because of a deeply disappointing experience w...,1
11,10 a week membership and not always the best v...,1
12,1m lost on straw trading to which all members ...,1


In [35]:
df.shape

(1404, 2)

In [36]:
df.reset_index(inplace=True)

In [37]:
df.head().tail()

,index,comment,vader_result
0,0,o been waiting for 9 months for one of your su...,2
1,1,useful admin support competitive purchasing pa...,0
2,10,because of a deeply disappointing experience w...,1
3,11,10 a week membership and not always the best v...,1
4,12,1m lost on straw trading to which all members ...,1


In [38]:
df.drop('index',axis=1, inplace=True)

In [39]:
df.head()

,comment,vader_result
0,o been waiting for 9 months for one of your su...,2
1,useful admin support competitive purchasing pa...,0
2,because of a deeply disappointing experience w...,1
3,10 a week membership and not always the best v...,1
4,1m lost on straw trading to which all members ...,1


In [40]:
df.reset_index(inplace=True)

In [41]:
df.drop('index',axis=1,inplace=True)

In [42]:
### Vocabulary size
voc_size=5000

In [43]:
y=df['vader_result']

In [44]:
y.value_counts()

0    1031
2     207
1     166
Name: vader_result, dtype: int64

In [45]:
X=df.drop('vader_result',axis=1)

In [46]:
messages=X.copy()

In [47]:
messages['comment'][1]

'useful admin support competitive purchasing particularly on chem fert'

In [48]:
import nltk
import re
from nltk.corpus import stopwords

In [49]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['comment'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [50]:
corpus

['wait month one supplier instal gate',
 'use admin support competit purchas particularli chem fert',
 'deepli disappoint experi deal spencer hill fuel trade supposedli member orient organis attitud deal error made depart extrem non custom friendli inflex rude discourt last email made angri never repli fear might said also copi complaint chief execut time receiv reaction whatsoev unforgiv past recommend friend sinc interact advis potenti exist custom acquaint tread extrem care deal af consid membership shame continu receiv excel servic other especi robert georg colleagu telecommun',
 'week membership alway best valu fee taken account',
 'lost straw trade member liabl',
 'big save fuel fuel card af take work find best deal telecom broadband landlin mobil good especi touch system build suppli enabl us make big save materi',
 'rate af consid renew membership',
 'time realli happi servic price achiev',
 'bit like nike',
 'first class servic make life work farmer much easier',
 'friendli te

In [63]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

TypeError: 'OneHotEncoder' object is not callable

In [88]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  497 1830 2860]
 [   0    0    0 ... 2739 2711 1494]
 [ 469 3157 2017 ... 2222 4926 3614]
 ...
 [   0    0    0 ... 1829 1527 2028]
 [   0    0    0 ...    0 4118 4211]
 [   0    0    0 ... 4591 1074 3637]]


In [89]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,  689,  732, 2323,  497, 1830, 2860])

In [90]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_3 (LSTM)               (None, 100)               56400     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [91]:
from tensorflow.keras.layers import Bidirectional
from keras.layers import Dropout

In [92]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional_2 (Bidirectio  (None, 200)              112800    
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [93]:
len(embedded_docs),y.shape

(1404, (1404,))

In [94]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [95]:
X_final.shape,y_final.shape

((1404, 20), (1404,))

In [96]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [97]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
15/15 [==============================] - 5s 88ms/step - loss: 0.6879 - accuracy: 0.7170 - val_loss: 0.6757 - val_accuracy: 0.7306
Epoch 2/10
15/15 [==============================] - 0s 26ms/step - loss: 0.6641 - accuracy: 0.7362 - val_loss: 0.6521 - val_accuracy: 0.7371
Epoch 3/10
15/15 [==============================] - 0s 26ms/step - loss: 0.5945 - accuracy: 0.6681 - val_loss: 0.5918 - val_accuracy: 0.6853
Epoch 4/10
15/15 [==============================] - 0s 26ms/step - loss: 0.2085 - accuracy: 0.7160 - val_loss: 0.4128 - val_accuracy: 0.6875
Epoch 5/10
15/15 [==============================] - 0s 26ms/step - loss: -0.3157 - accuracy: 0.7564 - val_loss: 0.3124 - val_accuracy: 0.6573
Epoch 6/10
15/15 [==============================] - 0s 26ms/step - loss: -0.8518 - accuracy: 0.7819 - val_loss: 0.3715 - val_accuracy: 0.7155
Epoch 7/10
15/15 [==============================] - 0s 27ms/step - loss: -1.6154 - accuracy: 0.7734 - val_loss: 0.2113 - val_accuracy: 0.6767
Epoch 8/10

In [98]:
y_pred1=model1.predict(X_test)

15/15 [==============================] - 1s 5ms/step


In [99]:
from sklearn.metrics import confusion_matrix

In [100]:
confusion_matrix(y_test,y_pred1.round())

array([[227, 112,   0],
       [ 17,  41,   0],
       [  9,  58,   0]], dtype=int64)

In [101]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1.round())

0.5775862068965517

In [128]:
filename = 'nlppp_model.pkl'
pickle.dump(model1, open(filename, 'wb'))

INFO:tensorflow:Assets written to: ram://dfd830c3-f1e3-41f9-8a82-ca20defefb7b/assets


INFO:tensorflow:Assets written to: ram://dfd830c3-f1e3-41f9-8a82-ca20defefb7b/assets


In [102]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1.round()))

              precision    recall  f1-score   support

           0       0.90      0.67      0.77       339
           1       0.19      0.71      0.30        58
           2       0.00      0.00      0.00        67

    accuracy                           0.58       464
   macro avg       0.36      0.46      0.36       464
weighted avg       0.68      0.58      0.60       464



C:\Users\mohan-prakashj\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\mohan-prakashj\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\mohan-prakashj\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [103]:
from flask import Flask,render_template,url_for,request
import pandas as pd
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import pickle

In [104]:
df.head()

,comment,vader_result
0,o been waiting for 9 months for one of your su...,2
1,useful admin support competitive purchasing pa...,0
2,because of a deeply disappointing experience w...,1
3,10 a week membership and not always the best v...,1
4,1m lost on straw trading to which all members ...,1


In [106]:
X = df['comment']
y = df['vader_result']

In [107]:
# Extract Feature With CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(X)

In [108]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y,
                                  test_size=0.25, random_state=42)

In [109]:
#Naive Bayes Classifier
from sklearn.naive_bayes import MultinomialNB

In [110]:
clf = MultinomialNB()
clf.fit(Xtrain,ytrain)
clf.score(Xtest,ytest)

0.7492877492877493

In [111]:
df.head()

,comment,vader_result
0,o been waiting for 9 months for one of your su...,2
1,useful admin support competitive purchasing pa...,0
2,because of a deeply disappointing experience w...,1
3,10 a week membership and not always the best v...,1
4,1m lost on straw trading to which all members ...,1


In [112]:
df.to_csv('nlpp.csv')

In [113]:
filename='nlp_model.pkl'

In [114]:
import pickle

In [115]:
pickle.dump(clf,open(filename,'wb'))

In [116]:
pickle.dump(cv, open('transform.pkl','wb'))

In [117]:
df.head()

,comment,vader_result
0,o been waiting for 9 months for one of your su...,2
1,useful admin support competitive purchasing pa...,0
2,because of a deeply disappointing experience w...,1
3,10 a week membership and not always the best v...,1
4,1m lost on straw trading to which all members ...,1


In [120]:
df.rename(columns={'comment':'message','vader_result':'label'},inplace=True)

In [125]:
df.head()

,message,label
0,o been waiting for 9 months for one of your su...,0
1,useful admin support competitive purchasing pa...,0
2,because of a deeply disappointing experience w...,1
3,10 a week membership and not always the best v...,1
4,1m lost on straw trading to which all members ...,1


In [124]:
df['label'] = df['label'].replace(2,0)

In [126]:
	X = df['message']
	y = df['label']
	
	# Extract Feature With CountVectorizer
	cv = CountVectorizer()
	X = cv.fit_transform(X) # Fit the Data
pickle.dump(cv, open('tranform.pkl', 'wb'))
   
    
	from sklearn.model_selection import train_test_split
	X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
	#Naive Bayes Classifier
	from sklearn.naive_bayes import MultinomialNB

	clf = MultinomialNB()
	clf.fit(X_train,y_train)
	clf.score(X_test,y_test)
filename = 'nlp_model.pkl'
pickle.dump(clf, open(filename, 'wb'))

In [127]:
df.to_csv('nlp.csv')

In [129]:
df.head()

,message,label
0,o been waiting for 9 months for one of your su...,0
1,useful admin support competitive purchasing pa...,0
2,because of a deeply disappointing experience w...,1
3,10 a week membership and not always the best v...,1
4,1m lost on straw trading to which all members ...,1


In [132]:
#split features and target from  data 
X = df["message"]
y = df.label.values

In [130]:
y = df.label.values

In [133]:
# split data into train and validate
X_train, X_valid, y_train, y_valid = train_test_split(
    X,
    y,
    test_size=0.15,
    random_state=42,
    shuffle=True,
    stratify=y,
)

In [135]:
# import important modules
import numpy as np
import pandas as pd
# sklearn modules
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB # classifier 
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    plot_confusion_matrix,
)
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [136]:
# Create a classifier in pipeline
sentiment_classifier = Pipeline(steps=[
                               ('pre_processing',TfidfVectorizer(lowercase=False)),
                                 ('naive_bayes',MultinomialNB())
                                 ])

In [137]:
# train the sentiment classifier 
sentiment_classifier.fit(X_train,y_train)

Pipeline(steps=[('pre_processing', TfidfVectorizer(lowercase=False)),
                ('naive_bayes', MultinomialNB())])

In [138]:
# test model performance on valid data 
y_preds = sentiment_classifier.predict(X_valid)

In [139]:
accuracy_score(y_valid,y_preds)

0.8815165876777251

In [140]:
#save model 
import joblib 
joblib.dump(sentiment_classifier, 'sentiment_model_pipeline.pkl')

['sentiment_model_pipeline.pkl']

In [146]:
filename = 'sentiment_model_pipeline.pkl'
pickle.dump(clf, open(filename, 'wb'))

In [145]:
import streamlit as st
from transformers import pipeline

option = st.selectbox(
    "Select an Option", 
    ["Sentiment Analysis",])

if option == "Sentiment Analysis":
    text = st.text_area(label="Enter text")
    if text: 
        classifier = pipeline("sentiment-analysis")
        answer = classifier(text)
        st.write(answer)

  command:

    streamlit run C:\Users\mohan-prakashj\Anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2022-07-18 12:21:47.617 
  command:

    streamlit run C:\Users\mohan-prakashj\Anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [144]:
!pip install transformers

     ---------------------------------------- 4.4/4.4 MB 6.7 MB/s eta 0:00:00
     ---------------------------------------- 3.3/3.3 MB 6.3 MB/s eta 0:00:00
     -------------------------------------- 101.5/101.5 kB 6.1 MB/s eta 0:00:00


In [183]:
df.head()

,comment,vader_result
0,o been waiting for 9 months for one of your su...,Neutral
1,useful admin support competitive purchasing pa...,Positive
10,because of a deeply disappointing experience w...,Negative
11,10 a week membership and not always the best v...,Negative
12,1m lost on straw trading to which all members ...,Negative


In [184]:
# Features & Labels
Xfeatures = df['comment']
ylabels = df['vader_result']

In [185]:
#Split Data
x_train,x_test,y_train,y_test = train_test_split(Xfeatures,ylabels,test_size=0.3,random_state=42)

In [186]:
# Build Pipeline
from sklearn.pipeline import Pipeline

In [187]:
# Load ML Pkgs
# Estimators
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

# Transformers
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [188]:
# LogisticRegression Pipeline
pipe_lr = Pipeline(steps=[('cv',CountVectorizer()),('lr',LogisticRegression())])

In [189]:
# Train and Fit Data
pipe_lr.fit(x_train,y_train)

Pipeline(steps=[('cv', CountVectorizer()), ('lr', LogisticRegression())])

In [190]:
pipe_lr

Pipeline(steps=[('cv', CountVectorizer()), ('lr', LogisticRegression())])

In [191]:
# Check Accuracy
pipe_lr.score(x_test,y_test)

0.8151658767772512

In [202]:
# Make A Prediction
ex1 = "they provide awfull customer service"

In [203]:
pipe_lr.predict([ex1])

array(['Positive'], dtype=object)

In [204]:
# Prediction Prob
pipe_lr.predict_proba([ex1])

array([[0.02643725, 0.31561003, 0.65795271]])

In [169]:
# To Know the classes
pipe_lr.classes_

array([0, 1], dtype=int64)

In [ ]:
from nltk.tag import post_tag
from nltk.corpus  import twitter_samples
from nlk.stem.wordnet import wordnetlemmatizer


def lemmatize_sentence(tokens):
    lemmatizer = wordnetlemmatizer()
    lemmatized_sentence=[]
    
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos='n'
        elif tag.startswith

In [29]:
df.head()

,comment,vader_result
0,o been waiting for 9 months for one of your su...,2
1,useful admin support competitive purchasing pa...,1
2,because of a deeply disappointing experience w...,0
3,10 a week membership and not always the best v...,0
4,1m lost on straw trading to which all members ...,0


In [17]:
df.reset_index(inplace=True)

In [20]:
df.drop('index',axis=1,inplace=True)

In [26]:
import pandas as pd
import numpy as np
import pickle
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer #For stemming
from nltk.stem import WordNetLemmatizer #For Lemmetization
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [28]:
df['vader_result'] = df['vader_result'].replace('Neutral',2).replace('Positive',1).replace('Negative',0)

In [30]:
df.shape

(1404, 2)

In [31]:
df_reviews_subset = df.iloc[:1000,:]

In [32]:
df_reviews_subset

,comment,vader_result
0,o been waiting for 9 months for one of your su...,2
1,useful admin support competitive purchasing pa...,1
2,because of a deeply disappointing experience w...,0
3,10 a week membership and not always the best v...,0
4,1m lost on straw trading to which all members ...,0
...,...,...
995,overall we get a good service from af but have...,1
996,overall you give a very good service hence fee...,1
997,paying a hefty yearly membership fee for servi...,2
998,people service efficiency easy bills,1


In [33]:
#get the stopwords for english
stopWords = stopwords.words('english')


In [34]:
#remove no and not from list of stopwords
stopWords.remove('no')
stopWords.remove('not')

In [35]:
#Create objects of stemmer and lemmatizer
stem_obj = PorterStemmer()
lem_obj = WordNetLemmatizer()

In [36]:
#Text Preprocessing
corpus = []

for i in range(0,len(df_reviews_subset)):
    review = re.sub('[^a-zA-Z]',' ',df_reviews_subset.iloc[i,0]) #remove all characters except A-Z
    review = review.lower().split() #convert all text into lower case and split by space
    
    #Apply stemming and lemmatization of all words in review
    review_updated = [lem_obj.lemmatize(lem_obj.lemmatize(stem_obj.stem(word),pos='v'),pos='a')\
                     for word in review\
                     if word not in stopWords]
    review_updated = ' '.join(review_updated)
    corpus.append(review_updated)

In [37]:
#initiliaze Countvecorizer
countVector = CountVectorizer(max_features=1500)

In [38]:
X = countVector.fit_transform(corpus).toarray()
y = df_reviews_subset.iloc[:,1].values

In [40]:
#split the dataset into train and test
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42) 

In [41]:
#Create Random forest model. Train and test
rf_classifer = RandomForestClassifier(n_estimators=150,min_samples_split=5)
rf_classifer.fit(X_train,y_train)
rf_classifer.score(X_test,y_test)

0.8266666666666667

In [42]:
#Save the model pickle file to be used for prediction
pickle.dump(rf_classifer,open('nlp_model.pkl','wb'))

In [43]:
pickle.dump(countVector,open('transform.pkl','wb'))

In [44]:
words = countVector.get_feature_names()
pickle.dump(words,open('vocabulary.pkl','wb'))